In [3]:
import os
if not os.path.exists('./jw_utils'):
    !git clone https://github.com/JonWinkelman/jw_utils.git
from orthofinder_utils import dash_app_preprocess as dap
from orthofinder_utils import proteomes_for_orthofinder as pfo
from orthofinder_utils import dash_ortho_parser_d as dop
from jw_utils import parse_gff as pgf
import zipfile
import pandas as pd
import json
from Bio import Phylo
import shutil
import plotly.graph_objects as go
from jw_utils import ncbi_datasets_fxs as nfx
from jw_utils import jw_ncbi_taxonomy as jnt


colors = {
    't_blue': 'rgba(0,102,153,255)',
    't_green': 'rgba(61,174,43,255)',
    't_red': 'rgb(255,20,20)',
    'seagreen':'#2c8d42',
    'orange':'#F9B257'}

In [4]:
summary_dir = './data/summary_data/'
if not os.path.exists(summary_dir):
    os.makedirs(summary_dir)
!datasets download genome accession --inputfile ./accessions.txt --include gff3,protein,genome
!datasets summary genome accession --inputfile ./accessions.txt > ./data/summary_data/summaries.json


In [13]:
zip_file_path = 'ncbi_dataset.zip'
extract_to_dir = 'data/ncbi_dataset'
os.makedirs(extract_to_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

print(f"Extracted all files to {extract_to_dir}")
shutil.move('./data/ncbi_dataset/ncbi_dataset/data','./data/ncbi_dataset/')
!rm -r ./data/ncbi_dataset/ncbi_dataset

FileNotFoundError: [Errno 2] No such file or directory: 'ncbi_dataset.zip'

In [5]:
data_dir = './data/ncbi_dataset/data/'
proteome_dir = 'data/Proteomes'
os.makedirs(proteome_dir, exist_ok=True)
for dir in os.listdir(data_dir):
    if dir.startswith('GC'):
        fs = os.listdir(os.path.join(data_dir,dir))
        for f in fs:
            if f.endswith('.faa'):
                new_f = (f'{dir}.faa')
                fp = os.path.join(data_dir,dir,f)
                new_fp = os.path.join(proteome_dir,new_f)
                print(fp)
                shutil.move(fp,new_fp )


### Setup for starting a orthofinder dash app    
***This is assuming you have run orthofinder on a set of proteomes and have the resulting    orthofinder data in a folder called Proteomes  
1. CHANGE WORKING DIRECTORY of this notebook TO dash_app main folder   
2. make a directory inside the main /dash_app folder called data (dash_app/data).
    - `$ !mkdir ./data`
    - `$ !mkdir ./data/Proteomes`
    - `$ !mv ./Proteomes/* ./data/Proteomes`

3. Place the ncbi_dataset/data containing all the data downloaded from ncbi into ./data
    - dash_app/data/ncbi_datasets/data  
4. You will need to generate a summary file of all genomes. Using the terminal is the most   
reliable way I've found to do this.  
    - `$ !datasets summary genome accession --inputfile accessions.txt > summaries.json`
 
 4) Run `dap.run_all()` in the cell below. If you haven't generated a summary file  
 then an error will be thrown, and code to get summary file via ncbi datasets cli will be  printed out


### Check that species in different directories are equivalent

In [6]:
def check_for_equivalent_species():
    if not os.path.exists('./data/summary_data/'):
        os.makedirs('./data/summary_data/')
    if not os.path.exists('./accessions.txt'):
        raise FileNotFoundError('"./accessions.txt" could not be found.')
    proteomes = [f[:15] for f in os.listdir('./data/Proteomes') if f.endswith('.faa')]
    assemblies = [f[:15] for f in os.listdir('./data/ncbi_dataset/data') if f.startswith('GC')]
    with open('./accessions.txt', 'r') as f:
        accessions = [line.strip() for line in f]
    if not sorted(accessions) == sorted(assemblies):
        raise Exception(f'"./data/ncbi_dataset/data" and "./accessions.txt" are not equivalent') 
    elif not sorted(assemblies) == sorted(proteomes):
        raise Exception(f'"./data/ncbi_dataset/data" and "/data/Proteomes" are not equivalent')
    elif not sorted(accessions) == sorted(proteomes):
        raise Exception(f'"./accessions.txt" and "/data/Proteomes" are not equivalent')
    else:
        print('All directories contain equivalent species')
    shutil.copy('./accessions.txt', './data/summary_data/accessions.txt')
    
accs_p = [a[:15] for a in os.listdir('./data/Proteomes/') if a.startswith('GC')]
acc_assemblies = [f[:15] for f in os.listdir('./data/ncbi_dataset/data') if f.startswith('GC')]
with open('./accessions.txt', 'r') as f:
    accs_a = []
    for line in f: 
        accs_a.append(line.strip())
with open('./accessions.txt', 'w') as f:
    accs_a = list(set(accs_a))
    for acc in accs_a:
        f.write(f'{acc}\n')



for acc in accs_p:
    if acc not in accs_a:
        print(f'Proteome {acc} not in ./accessions.txt')
for acc in accs_a:
    if acc not in accs_p:
        print(f'Assession {acc} from ./assessions.txt  not in ./data/Proteomes/')

In [7]:
check_for_equivalent_species()


All directories contain equivalent species


In [38]:
!orthofinder -f ./data/Proteomes

/bin/bash: orthofinder: command not found


## Generate data files for dash app
### !!Change path for latest orthofinder results!!

In [37]:
if not os.path.exists('./data/Proteomes/OrthoFinder'):
    os.mkdir('./data/Proteomes/OrthoFinder')

In [30]:
import importlib
importlib.reload(dap)
path_to_parent=os.getcwd()
path_to_results = './data/Proteomes/OrthoFinder/Results_Aug26'
dap.run_all(path_to_results)

### Make tree annotation file

In [31]:
sp_tree_path = './data/Species_Tree/SpeciesTree_rooted_node_labels.txt'
path_to_summary ='./data/summary_data/summaries.json'

def make_internal_node_d(tree):
    """"""
    internal_node_dict = {}
    for node in tree.get_nonterminals():
        internal_node_dict[node.name] = {'name':node.name, 'rank':'', 'taxid':'', 'sci_name':''}
    for node in tree.get_terminals():
            internal_node_dict[node.name] = {'name':node.name}
    internal_node_dict

    with open('./data/summary_data/internal_node_dict.json', 'w') as f:
        json.dump(internal_node_dict, f)
    internal_node_dict

make_internal_node_d(Phylo.read(sp_tree_path, format='newick'))

### Make node rank lineage file

In [33]:
# python3 app1.py


/Library/Frameworks/Python.framework/Versions/3.10/bin/python3: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3: cannot execute binary file
